In [30]:
import pandas as pd
from tqdm import tqdm
import json
import requests
from pathlib import Path
import re
import random
import numpy as np
import time

In [31]:
def get_publisher_download_url(crossref_resp):
    # search for 'text-mining' in the intended-application field
    text_mining = []
    for link in crossref_response.json()['link']:
        if re.search(r'text-mining',link['intended-application']):
            text_mining.append(re.search('text-mining', link['intended-application']))
        else:
            text_mining.append(None)
            
    # find the index of match obects in crossref link information
    publisher_api_idxs = np.where(np.array(text_mining) != None)[0].tolist()
    # wiley_api_idxs
    links = [crossref_response.json()['link'][i] for i in publisher_api_idxs]
    # for now use the first link if there are more than one link
    full_text_link = links[0]['URL']
    return full_text_link

In [ ]:
def download_pdf(doi, dest_folder, content):
    doi_suffix = str(doi.split('/', 1)[1:]).strip("[']")
    doi_suffix = re.sub(r"""([()/\\*,"': ?;<>])""", '_._', doi_suffix)
    filename = dest_folder + str(doi_suffix)+'.pdf'
    with open(filename, mode = 'wb') as f:
        f.write(content)

In [ ]:
doi = 'eurjhf/hfp181.pdf'
doi_suffix = str(doi.split('/', 1)[1:]).strip("[']")
re.sub(r"""([()\\*,"': ?;<>])""", '_._'
filename = dest_folder + str(doi_suffix)+'.pdf'
with open(filename, mode = 'wb') as f:
    f.write(content)

In [32]:
with open('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/code/cpet_articles/gathering/full-text_download_code/wiley/wiley_config.json') as config_file:
    wiley_token = json.load(config_file)['api_key']

In [33]:
pdf_file_paths = list(Path('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/full_texts/pdfs').rglob('*/*.pdf'))
pdfs = [path.stem for path in pdf_file_paths]
len(pdfs)

12825

In [34]:
articles = pd.read_csv('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/unpaywall/unpaywall_info.csv')
wiley_oa_articles = articles[(articles['publisher'] == 'Wiley') & (articles['is_oa'] == True)].drop_duplicates().reset_index(drop=True)

In [35]:
re_doi_suffix = re.compile(r'(?<=\d/).*')
wiley_oa_articles['doi_suffix'] = wiley_oa_articles['doi'].apply(lambda x: re_doi_suffix.search(x).group())

In [36]:
full_texts_to_download = [x for x in wiley_oa_articles['doi_suffix'].tolist() if x not in pdfs]
merge = pd.merge(pd.DataFrame({'doi_suffix': full_texts_to_download}), wiley_oa_articles, how='inner', on='doi_suffix')
merge.shape

(496, 50)

In [42]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:101.0) Gecko/20100101 Firefox/101.0'
accept = 'application/vnd.citationstyles.csl+json, application/vnd.crossref.unixref+xml'
dest_folder = '/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/full_texts/pdfs/wiley_oa'

In [43]:
# n = random.randint(0, merge.shape[0])
# row = merge.loc[n,:]
# doi = row['doi']
# doi

In [44]:
# doi = row['doi']
# out = {'doi': doi}
# doi_url = 'https://doi.org/' + str(doi)
# crossref_headers = {'Accept': accept, 'User-Agent': user_agent}
# try:
#     crossref_response = requests.get(
#                     url = doi_url,
#                     headers=crossref_headers,
#                     allow_redirects=True,
#                     verify=True)
#     out.update({'CR_status_code': crossref_response.status_code})

#     if crossref_response.status_code == 200:
#         full_text_url = get_publisher_download_url(crossref_response)
#         publisher_headers = {
#             'User-Agent': user_agent,
#             'Wiley-TDM-Client-Token': wiley_token,
#             # 'CR-TDM-Rate-Limit': '1000',
#             # 'CR-TDM-Rate-Limit-Remaining': '76',
#             # 'CR-TDM-Rate-Limit-Reset': '1378072800'
#         }
#         publisher_response = requests.get(full_text_url, headers = publisher_headers, allow_redirects=True)
#         out.update({'publisher_status_code': publisher_response.status_code})
#         if publisher_response.status_code == 200:
#             doi_suffix = str(doi.split('/', 1)[1:]).strip("[']")
#             filename = f'{dest_folder}/{doi_suffix}.pdf'
#             with open(filename, mode='wb') as f:
#                 f.write(publisher_response.content)
# except Exception as e:
#     print(e)
#     out.update({'error': e})

# print(publisher_response.status_code)

In [45]:
log = []

In [ ]:
for idx, row in tqdm(merge.iterrows(), total=merge.shape[0]):
    doi = row['doi']
    out = {'doi': doi}
    doi_url = 'https://doi.org/' + str(doi)
    crossref_headers = {'Accept': accept, 'User-Agent': user_agent}
    try:
        crossref_response = requests.get(
                        url = doi_url,
                        headers=crossref_headers,
                        allow_redirects=True,
                        verify=True)
        out.update({'CR_status_code': crossref_response.status_code})

        if crossref_response.status_code == 200:
            full_text_url = get_publisher_download_url(crossref_response)
            publisher_headers = {
                'User-Agent': user_agent,
                'Wiley-TDM-Client-Token': wiley_token,
                # 'CR-TDM-Rate-Limit': '1000',
                # 'CR-TDM-Rate-Limit-Remaining': '76',
                # 'CR-TDM-Rate-Limit-Reset': '1378072800'
            }
            publisher_response = requests.get(full_text_url, headers = publisher_headers, allow_redirects=True)
            out.update({'publisher_status_code': publisher_response.status_code})
            if publisher_response.status_code == 200:
                download_pdf(doi = doi, dest_folder=dest_folder, content=full_text_resp.content)
    except Exception as e:
        print(e)
        out.update({'error': e})
    time.sleep(0.35)

    log.append(out)

  4%|▍         | 19/496 [00:15<07:40,  1.04it/s]

[Errno 2] No such file or directory: '/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/full_texts/pdfs/wiley_oa/eurjhf/hfq086.pdf'


  4%|▍         | 21/496 [00:21<13:55,  1.76s/it]

[Errno 2] No such file or directory: '/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/full_texts/pdfs/wiley_oa/eurjhf/hfp181.pdf'


  5%|▍         | 23/496 [00:25<14:25,  1.83s/it]

...

 59%|█████▉    | 294/496 [09:53<03:04,  1.09it/s]

HTTPConnectionPool(host='api.wiley.com', port=80): Max retries exceeded with url: /onlinelibrary/tdm/v1/articles/10.1113%2Fjphysiol.1987.sp016420 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f8163087550>: Failed to establish a new connection: [Errno 61] Connection refused'))


 60%|█████▉    | 296/496 [09:55<02:59,  1.12it/s]

HTTPConnectionPool(host='api.wiley.com', port=80): Max retries exceeded with url: /onlinelibrary/tdm/v1/articles/10.1111%2Fj.1365-2125.1981.tb01205.x (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f816309b7c0>: Failed to establish a new connection: [Errno 61] Connection refused'))


 60%|██████    | 300/496 [09:58<02:43,  1.20it/s]

In [ ]:
log_df = pd.DataFrame(log)

In [280]:
non200 = log_df[log_df['publisher_status_code'] != 200].reset_index(drop=True)
non200.shape

In [281]:
wiley_errors = pd.merge(non200, merge, how='inner', on='doi')
wiley_errors.to_csv('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/unpaywall/wiley_oa_status_codes.csv',
                    index=False)

In [283]:
wiley_errors.shape

(693, 53)